In [98]:
!python3 -m pip install pytesseract
!python3 -m pip install bs4


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from piazza_api import Piazza
import json
import time
from collections import defaultdict

#Imports for OCR
import pytesseract
import io
import requests
from bs4 import BeautifulSoup
try:
    from PIL import Image
except ImportError:
    import Image


In [3]:
p = Piazza()
p.user_login()

In [4]:
user_profile = p.get_user_profile()

In [5]:
course_key_dict = {}
course_key_dict['spring_2022_nlp'] = 'ky7ls2h92kpwe'
course_key_dict['fall_2022_nlp'] = 'l7102doc7aa3ob'
course_key_dict

{'spring_2022_nlp': 'ky7ls2h92kpwe', 'fall_2022_nlp': 'l7102doc7aa3ob'}

In [6]:
regular_posts = defaultdict(list)
image_posts = defaultdict(list)

In [7]:
def get_posts(key, post_ids, mapper=None, **kwargs):
    all_posts = []
    if key in course_key_dict:
        #get course
        course = p.network(course_key_dict[key])

        #get post from course object
        for i in range(len(post_ids)):
            try:
                post = course.get_post(post_ids[i])
                all_posts.append(post)
                time.sleep(2)
            except Exception as e:
                print("Error ", i, e)
        

        #map the post if required
        if mapper:
            all_posts = [mapper(post, **kwargs) for post in all_posts]
        
    return all_posts

In [8]:
def image_to_string(url):
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    extractedInformation = pytesseract.image_to_string(img)
    return extractedInformation

def get_image_urls(html):
    urls = []
    soup = BeautifulSoup(html)
    for link in soup.findAll('img'):
        urls.append(link.get('src'))
    return urls

def add_question_link(post, **kwargs):
    if post and 'course_name' in kwargs and kwargs['course_name'] in course_key_dict:
        post['question_link'] = "https://piazza.com/class/" + course_key_dict[kwargs['course_name']] + "/post/" + str(post['nr'])
    #"question_link": "https://piazza.com/class/l7102doc7aa3ob/post/689"
    return post



def image_to_question(post, **kwargs):
    add_question_link(post, **kwargs)
    for history in post['history']:
        urls = get_image_urls(history['content'])
        urls = ["https://piazza.com" + x for x in urls]
    
        questions = []
        for i in range(len(urls)):
            questions.append(image_to_string(urls[i]))
    
        question_string = "\n".join(questions)
        history['content_orig'] = history['content']
        history['content'] = question_string
    
    return post

def write_to_file(posts, file_name):
    def obj_dict(obj):
        return obj.__dict__
    json_string = json.dumps(posts, indent=4)
    f = open('data/' + file_name + '.json', 'wb')
    f.write(json_string.encode())


In [12]:
regular_posts['spring_2022_nlp'] = [871,870,868,594,590,576,570,544,487,382,268,255,251,243,103,54,18]
regular_posts['fall_2022_nlp'] = [689,663,638,608,593,579,541,536,589,529,528,511,506,497,490,484,443,348,254,245,241,227,85,84,79,58,39,20,22,37,86,101,116,317,347,358,362]

image_posts['spring_2022_nlp'] =  [871, 870, 868, 861]
image_posts['fall_2022_nlp'] = [627]

In [13]:
regular_posts

defaultdict(list,
            {'spring_2022_nlp': [871,
              870,
              868,
              594,
              590,
              576,
              570,
              544,
              487,
              382,
              268,
              255,
              251,
              243,
              103,
              54,
              18],
             'fall_2022_nlp': [689,
              663,
              638,
              608,
              593,
              579,
              541,
              536,
              589,
              529,
              528,
              511,
              506,
              497,
              490,
              484,
              443,
              348,
              254,
              245,
              241,
              227,
              85,
              84,
              79,
              58,
              39,
              20,
              22,
              37,
              86,
              101,
              116,
      

In [10]:
for key, val in image_posts.items():
    posts = get_posts(key, val, image_to_question, course_name=key)
    write_to_file(posts, 'image_' + key)

In [14]:
for key, val in regular_posts.items():
    posts = get_posts(key, val, add_question_link, course_name=key)
    write_to_file(posts, key)